Adapted from `rlcard/examples/leduc_holdem_dqn_pytorch.py`

In [ ]:
!git clone https://github.com/DanielKerrigan/CS-GY_9223_Deep_Learning.git

%cd CS-GY_9223_Deep_Learning

!git clone https://github.com/DanielKerrigan/rlcard.git

%cd rlcard

!git checkout dan

%cd ..

from google.colab import drive
drive.mount('/content/gdrive')
drive_dir = '/content/gdrive/My Drive/deep-learning'

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath('./rlcard'))

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
import torch
import rlcard
from DQNAgent import DQNAgent
from rlcard.agents import RandomAgent
from rlcard.utils import set_global_seed, tournament
from rlcard.utils import Logger

In [ ]:
# Make environment
env = rlcard.make('limit-holdem', config={'seed': 0})
eval_env = rlcard.make('limit-holdem', config={'seed': 0})

In [ ]:
# Set the iterations numbers and how frequently we evaluate the performance
evaluate_every = 20_000
evaluate_num = 10_000
episode_num = 1_000_000
save_every = 100_000

In [ ]:
# Set a global seed
set_global_seed(0)

In [ ]:
agent = DQNAgent(
    env.action_num,
    env.state_shape[0],
    memory=10_000_000,
    lr=0.1,
    batch_size=256,
    update_every=128,
    eps_start=0.12,
    target_update=500,
    hidden_neurons=[1024, 512, 1024, 512]
)

random_agent = RandomAgent(action_num=eval_env.action_num)
env.set_agents([agent, agent])
eval_env.set_agents([agent, random_agent])

In [ ]:
# Init a Logger to plot the learning curve
logger = Logger(drive_dir)

In [ ]:
for episode in range(episode_num):
    # Generate data from the environment
    trajectories, _ = env.run(is_training=True)

    # Feed transitions into agent memory, and train the agent
    for ts in trajectories[0]:
        agent.train(ts)

    # Evaluate the performance. Play with random agents.
    if episode % evaluate_every == 0:
        logger.log_performance(agent.weight_updates,
                               tournament(eval_env, evaluate_num)[0])
        
    if episode % save_every == 0:
        agent.save(drive_dir)

In [ ]:
# Close files in the logger
logger.close_files()

In [ ]:
# Plot the learning curve
logger.plot('DQN')

In [ ]:
eval_env_dqn_dqn = rlcard.make('limit-holdem', config={'seed': 1})
eval_env_dqn_dqn.set_agents([agent, agent])
payoffs = tournament(eval_env_dqn_dqn, 10000)
print(payoffs)